# Bluetooth Arm test

This code will test the arm over bluetooth, and also test kinematics on the arm.

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport dynamixel

In [ ]:
from arm import Arm, JointPathWaypoint
import kinematics
import time
import numpy as np
from util import SE3

In [ ]:
PORT = '/dev/tty.borgla_test'
BAUD = 9600
WRITE_PACKET_DELAY = 0.15  # This is because bluetooth is a bit slow
# PORT = '/dev/tty.usbmodem1101'
# BAUD = 9600
# WRITE_PACKET_DELAY = 0.01

Robot = lambda: Arm(PORT, BAUD, write_packet_delay=WRITE_PACKET_DELAY)

In [ ]:
# Test connection to all servos by pinging
# We expect print 6 lines, all with error=0.  See https://emanual.robotis.com/docs/en/dxl/protocol1/#error
with Robot() as robot:
    robot.ping(0)
    robot.ping(1)
    robot.ping(2)
    robot.ping(3)
    robot.ping(4)
    robot.ping(5)
    for msg in robot.read_all_msgs():
        print(msg)

In [ ]:
# Allow hand-positioning of arm and print joint angles
with Robot() as robot:
    robot.disable_all()
    while input() != 'q':
        print(robot.read_all_joint_angles_deg())

In [ ]:
# Set arm to 0 position
with Robot() as robot:
    robot.enable_all()
    robot.go_to_blocking([0, 0, 0, 0, 0])

In [ ]:
# Test following a path (dipping paint)
path = [
    JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),            # home
    JointPathWaypoint(q=[0, -65, 65, -1, 8], tol=5, timeout=None, pause=0),      # lower
    JointPathWaypoint(q=[37, -63, 122, -4.8, 78], tol=5, timeout=None, pause=0), # hover paint
    JointPathWaypoint(q=[37, -40, 124, -4.8, 78], tol=5, timeout=None, pause=0), # dip
    JointPathWaypoint(q=[37, -26, 120, -5.1, 78], tol=5, timeout=None, pause=0), # rub
    JointPathWaypoint(q=[37, -50, 130, -5.1, 78], tol=5, timeout=None, pause=0), # rub
    JointPathWaypoint(q=[37, -67, 130, -4.8, 78], tol=5, timeout=None, pause=0), # lift paint
    JointPathWaypoint(q=[0, -65, 65, -1, 8], tol=5, timeout=None, pause=0),      # raise
    JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),            # home
]
with Robot() as robot:
    robot.enable_all()
    robot.set_speed(50)
    robot.execute_joint_path(path, verbosity=0, default_speed=50)

In [ ]:
# Check FK by disabling motors and printing tip position as the robot is moved by hand
with Robot() as robot:
    robot.disable_all()
    print(f'{"Tip Position (m)":25}\t{"Joint Angles (deg)":30}')
    while True:
        try:
            q = np.array(robot.joint_angles_deg())
            wTtip, _ = kinematics.fk_deg(q)
            wPtip = wTtip[:3, 3]
            print(f'{str(wPtip.round(3)):25}\t{str(q.round(0)):30}', end='\r')
        except KeyboardInterrupt:
            break
    print()
    print(wTtip)

In [ ]:
with Robot() as robot:
    print(robot.set_compliance_slopes(16))
    time.sleep(0.5)
    print(robot.set_compliance_slopes(8))
    time.sleep(0.5)
    print(robot.set_compliance_slopes(4))
    time.sleep(0.5)
    print(robot.set_compliance_slopes(2))
    time.sleep(0.5)
    print(robot.set_compliance_slopes(0))
    time.sleep(0.5)
    robot.go_to_blocking([0, 0,0, 0, 0], tol=2, timeout=None)
    robot.go_to_blocking([0, -90, 90, 0, 0], tol=5, timeout=None)
    print(robot.set_compliance_slopes(16))
    time.sleep(0.5)
    print(robot.set_compliance_slopes(8))
    time.sleep(0.5)
    print(robot.set_compliance_slopes(4))
    time.sleep(0.5)
    print(robot.set_compliance_slopes(2))
    time.sleep(0.5)
    print(robot.set_compliance_slopes(0))
    time.sleep(0.5)
    print(robot.joint_angles_deg())

In [ ]:
with Robot() as robot:
    print(robot.joint_angles_deg())

In [ ]:
# Check IK by moving to the last point in the path above
wTgoal1 = SE3.P(wPtip) @ SE3.RotX(-np.pi/2)  # Brush tip position, but pointing at the glass
wTgoal2 = wTtip.copy()

success, q1 = kinematics.ik_deg(wTgoal1)
assert success, 'IK failed'
print('IK solution: ', q1.round(0))
success, q2 = kinematics.ik_deg(wTgoal2)
assert success, 'IK failed'
print('IK solution: ', q2.round(0))

path = [
    JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
    JointPathWaypoint(q=q1, tol=5, timeout=None, pause=0),               # goal
    JointPathWaypoint(q=q2, tol=5, timeout=None, pause=0),               # goal
    JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
]
with Robot() as robot:
    robot.enable_all()
    robot.set_speed(50)
    robot.execute_joint_path(path, verbosity=2)

In [ ]:
# Check IK by moving to the last point in the path above
# wTgoal1 = SE3.P(wPtip) @ SE3.RotX(-np.pi/2)  # Brush tip position, but pointing at the glass
wTgoals = [SE3.P(CENTER_POINT + [0, -0.1, 0.1]) @ SE3.RotX(-1.57 + i - .5) for i in np.linspace(0, 0.5, 5)]  # Brush tip position, but pointing at the glass

sols = [kinematics.ik_deg(wTgoal, elbow_mode='pos') for wTgoal in wTgoals]
assert all(success for success, _ in sols), 'IK failed'
qs = [q for _, q in sols]
for q in qs:
    print('IK solution: ', q.round(0))

path = [
    JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
    *[JointPathWaypoint(q=q, tol=5, timeout=None, pause=0) for q in qs],               # goal
    JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
]
with Robot() as robot:
    robot.enable_all()
    robot.set_speed(50)
    robot.execute_joint_path(path, verbosity=2)
    # robot.execute_joint_path(path[:-1], verbosity=2)
    # input()
    # robot.execute_joint_path(path[-1:], verbosity=2)

In [ ]:
with Robot() as robot:
    print(robot.set_compliance_margins(1))
    print(robot.set_compliance_slopes(16))

In [ ]:
with Robot() as robot:
    robot.disable_all()

In [ ]:
CENTER_POINT = np.array([-0.03205, 0.363, 0.24525])
MAX_VEL = 50

In [ ]:
# Finally, try drawing something simple, like a circle
wPc = CENTER_POINT

# t = np.linspace(0, 2 * np.pi, 10)[:, None]
# positions = wPc + np.hstack([np.cos(t), 0 * t, np.sin(t)]) * 0.03

N = 5
t = np.linspace(-1, 1, N)[:, None]
t = np.vstack([t[N // 2:0:-1], t, t[N - 2:N // 2 - 1:-1]])
print(t)
positions = wPc + np.hstack([0.1 * t, 0 * t, 0 * t])
# Ease in/out by adding a point to the beginning and end with y-=0.12
positions = np.vstack([positions[0], positions, positions[-1]])
# positions[0][1] = special_plane
# positions[-1][1] = special_plane
positions[0][1] -= 0.07
positions[-1][1] -= 0.07

print(positions)

Ts = [(SE3.P(p) @ SE3.RotX(-np.pi / 2)) for p in positions]
qs = [kinematics.ik_deg(T, elbow_mode='pos') for T in Ts]
assert all(q[0] for q in qs), f'IK failed: {[q[0] for q in qs]}'
qs = [q[1] for q in qs]
# Create velocities
qdots = [kinematics.joint_vels_deg(q1, q2, max_vel=MAX_VEL) for q1, q2 in zip(qs[:-1], qs[1:])]
qdots.insert(0, [MAX_VEL, 0.8*MAX_VEL, MAX_VEL, MAX_VEL, 0.8*MAX_VEL])
qdots = [np.clip(np.abs(qdot), 5, None) for qdot in qdots]
for q, qdot in zip(qs, qdots):
    print(q.round(0), qdot.round(0))
# qdots[0] = None

path = [
    JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
    *[JointPathWaypoint(q=q, speeds=qdot, tol=5, timeout=None, pause=0) for q, qdot in zip(qs, qdots)],  # trajectory
    JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
]
# path[2].pause = 1
# path[-3].pause = 1
with Robot() as robot:
    robot.enable_all()
    robot.set_speed(MAX_VEL)
    print(robot.set_compliance_slopes(32))
    robot.execute_joint_path(path, verbosity=2, default_speed=MAX_VEL)
    # robot.disable_all()

In [ ]:
def create_path(centerPxyz, vmax=MAX_VEL):
    # First, add the ease in/out
    ease_point = np.array([0, -0.05, 0.0])
    centerPxyz = np.vstack([ease_point + centerPxyz[0], centerPxyz, ease_point + centerPxyz[-1]])
    # Now turn positions into global poses
    wTxyz = [(SE3.P(p + CENTER_POINT) @ SE3.RotX(-np.pi / 2)) for p in centerPxyz]
    # Finally, convert to joint angles
    qs = [kinematics.ik_deg(T, elbow_mode='pos') for T in wTxyz]
    assert all(q[0] for q in qs), f'IK failed: {[q[0] for q in qs]}'
    qs = [q[1] for q in qs]
    assert all(q[-1] > -108 for q in qs), f'Joint limit reached: \n{np.array(qs)}'
    # Now, handle velocities
    qdots = [kinematics.joint_vels_deg(q1, q2, max_vel=vmax) for q1, q2 in zip(qs[:-1], qs[1:])]
    qdots.insert(0, [vmax, 0.8*vmax, vmax, vmax, 0.8*vmax])
    qdots = [np.clip(np.abs(qdot), vmax * 0.15, None) for qdot in qdots]
    # Finally, create and return the path
    return [
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
        *[JointPathWaypoint(q=q, speeds=qdot, tol=5, timeout=None, pause=0) for q, qdot in zip(qs, qdots)],  # trajectory
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
    ]

In [ ]:
t = np.linspace(0, 2 * np.pi, 9)[:, None]
positions = np.hstack([np.cos(t), 0 * t, np.sin(t) + .4]) * 0.08
path = create_path(positions, vmax=75)

with Robot() as robot:
    robot.enable_all()
    robot.set_speed(75)
    robot.set_compliance_slopes(32)
    robot.execute_joint_path(path, verbosity=2, default_speed=75)

In [ ]:
path = create_path(np.array([0, -0.05, 0.1])[None, :], vmax=75)

with Robot() as robot:
    robot.enable_all()
    robot.set_speed(75)
    robot.set_compliance_slopes(32)
    robot.execute_joint_path(path[:3], verbosity=2, default_speed=75)
    input()
    robot.execute_joint_path(path[3:], verbosity=2, default_speed=75)

In [ ]:
# Try tilting the brush

Ts = [(SE3.P(CENTER_POINT + [0, 0, 0.1]) @ SE3.RotX(-np.pi / 2 + angle)) for angle in np.linspace(0, 1, 5)]
qs = [kinematics.ik_deg(T, elbow_mode='pos') for T in Ts]
assert all(q[0] for q in qs), f'IK failed: {[q[0] for q in qs]}'
qs = [q[1] for q in qs]
assert all(q[-1] > -108 for q in qs), f'Joint limit reached: \n{np.array(qs)}'

# qdots = [kinematics.joint_vels_deg(q1, q2, max_vel=MAX_VEL) for q1, q2 in zip(qs[:-1], qs[1:])]
# qdots.insert(0, [MAX_VEL, 0.8*MAX_VEL, MAX_VEL, MAX_VEL, 0.8*MAX_VEL])
# qdots = [np.clip(np.abs(qdot), 5, None) for qdot in qdots]

path = [
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
        # *[JointPathWaypoint(q=q, speeds=qdot, tol=5, timeout=None, pause=0) for q, qdot in zip(qs, qdots)],  # trajectory
        *[JointPathWaypoint(q=q, tol=5, timeout=None, pause=0) for q in qs],  # trajectory
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
    ]

with Robot() as robot:
    robot.enable_all()
    robot.set_compliance_slopes(16)
    robot.execute_joint_path(path, verbosity=2, default_speed=MAX_VEL)